### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
# 使用 colab 環境的同學請執行以下程式碼
#%tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
 import tensorflow as tf
 print(tf.__version__)

 import os
 from google.colab import drive 
 drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
 %cd 'gdrive/My Drive/Colab Notebooks/D43/'
# os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
# %cd cupoy_cv_part4 # 可以自己改路徑

2.2.0-rc2
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/D43


In [0]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [5]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [0]:
from keras.models import *
#from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.layers import *

In [25]:
# 定義人臉關鍵點檢測網路

model = Sequential()

# 定義神經網路的輸入, hidden layer 以及輸出
input_tensor = Input((96,96,1))
x = input_tensor
for i in range(4):
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = BatchNormalization(axis=-1)(x)
    if i <3:
        x = MaxPooling2D(pool_size=(2, 2))(x)
    else:
        x = MaxPooling2D(pool_size=(2, 1))(x)

x=(Flatten())(x)
x=Dense(512, activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(512, activation='relu')(x)
x=Dropout(0.2)(x)

# 最後輸出 30 維的向量，也就是 15 個關鍵點的值
x = Dense(30)(x)
model = Model(input=input_tensor, output=x)

# 配置 loss funtion 和 optimizer
model.compile(loss='mean_squared_error', optimizer='adam')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [26]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 94, 94, 32)        320       
_________________________________________________________________
batch_normalization_49 (Batc (None, 94, 94, 32)        128       
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 92, 92, 32)        9248      
_________________________________________________________________
batch_normalization_50 (Batc (None, 92, 92, 32)        128       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 44, 44, 32)        9248